In [1]:
# nbi:hide_in
##########################
# Created on Mar 2020
# @author: juans
##########################

In [2]:
# nbi:hide_in
# Requried libraries
# %matplotlib notebook
%matplotlib inline
import numpy as np
import scipy.signal as sg
import matplotlib.pyplot as plt
import ipywidgets as iPyWid

In [3]:
# nbi:hide_in
def mag2db(x):
    return 20.0 * np.log10(x)

def db2mag(x):
    return 10.0**(x / 20)

def db(x):
    return mag2db(np.abs(x))

def deg(x):
    return np.rad2deg(np.angle(x))

In [4]:
# nbi:hide_in
def shelfFilter(freq, gainIndB, btype):
    if gainIndB == 0:
        return np.array([1]), np.array([1])
    wo = 2 * np.pi * freq
    g = db2mag(gainIndB)
    
    a1 = b1 = 1.0 / wo
    a0 = b0 = 1.0
    
    if btype == 'high' and gainIndB > 0:
        b1 *= g
    elif btype == 'high' and gainIndB < 0:
        a1 /= g
    elif btype == 'low' and gainIndB > 0:
        b0 *= g
    elif btype == 'low' and gainIndB < 0:
        a0 /= g
    
    return np.array([b1, b0]), np.array([a1, a0])
        
        
def parametricFilter(freq, gainIndB, Q):
    if (gainIndB == 0):
        return np.array([0, 0, 1]), np.array([0, 0, 1])
    wo = 2 * np.pi * freq
    g = db2mag(gainIndB)
    
    a2 = b2 = 1.0 / (wo**2)
    a1 = b1 = 1.0 / (wo*Q)
    a0 = b0 = 1.0
    
    if (gainIndB > 0):
        b1 *= g
    else:
        a1 /= g
        
    return np.array([b2, b1, b0]), np.array([a2, a1, a0])

def warpedBilinearTransform(b, a, fs, fp):
    z, p, k = sg.tf2zpk(b, a)
    wo = 2 * np.pi * fp
    c = wo / np.tan(wo / (2 * fs))
    
    zd = (1 + z/c)/(1 - z/c)
    pd = (1 + p/c)/(1 - p/c)
    zd = np.concatenate([zd, -np.ones(len(pd) - len(zd))])
    kd = k * np.real(np.prod(c - z) / np.prod(c - p))
    bd, ad = sg.zpk2tf(zd, pd, kd)
    return bd, ad

In [5]:
# nbi:hide_in
filtTypeDropdown = iPyWid.Dropdown(
    options=['LowPass', 'HighPass', 'HighShelf', 'LowShelf', 'Parametric'],
    value='Parametric',
    description='Filter Type'
)
    
    
fcSlider = iPyWid.FloatLogSlider(
    value=1e3,
    base=10,
    min=np.log10(2e1), # max exponent of base
    max=np.log10(2e4), # min exponent of base
    step=0.01, # exponent step
    description='freq',
    readout_format='d',
    continuous_update=False
)

gainSlider = iPyWid.IntSlider(
    value=1,
    min=-12,
    max=12,
    step=1,
    description='Gain',
    continuous_update=False
)

qSlider = iPyWid.FloatLogSlider(
    value=1,
    base=10,
    min=-1,
    max=1,
    step=0.1,
    description='Q',
    continuous_update=False
)

fsSlider = iPyWid.FloatLogSlider(
    value=44.1e3,
    base=10,
    min=np.log10(44100), 
    max=np.log10(192000), 
    step=0.001, 
    description='fs',
    readout_format='d',
    continuous_update=False
)

In [6]:
# nbi:hide_in
N = 200
fax = 2 * np.logspace(1, 4, N)

def plot(filtType, fc, gainIndB, Q, fs):

    if filtType == 'LowPass':
        b, a = sg.butter(2, 2 * np.pi * fc, btype='low', analog=True)
    elif filtType == 'HighPass':
        b, a = sg.butter(2, 2 * np.pi * fc, btype='high', analog=True)
    elif filtType == 'LowShelf':
        b, a = shelfFilter(fc, gainIndB, btype='low')
    elif filtType == 'HighShelf':
        b, a = shelfFilter(fc, gainIndB, btype='high')
    elif filtType == 'Parametric':
        b, a = parametricFilter(fc, gainIndB, Q)
    else:
        raise Exception('Filter Type not found')
        
    if (gainIndB == 0):
        bd = np.flip(b)
        ad = np.flip(a)
    else:
        bd, ad = warpedBilinearTransform(b, a, fs, fc)

    _, Ha = sg.freqs(b, a, 2 * np.pi * fax)
    _, Hd = sg.freqz(bd, ad, fax, fs=fs)
    plt.figure(figsize=(12, 9))
    plt.subplot(211)
    plt.semilogx(fax, db(Ha), label="Analog")
    plt.semilogx(fax, db(Hd), label="Digital")
    plt.semilogx([2e4, 2e4], [-12, 12])
    plt.grid(True)
    plt.xlim([2e1, 2e4])
    plt.ylim([-12, 12])
    plt.yticks(np.arange(-12, 12.1, 3))
    plt.ylabel('Amplitude (dB)')
    plt.title('Transfer Function')
    plt.legend()

    plt.subplot(212)
    plt.semilogx(fax, deg(Ha))
    plt.semilogx(fax, deg(Hd))
    plt.semilogx([2e4, 2e4], [-180, 180])
    plt.grid(True)
    plt.xlim([2e1, 2e4])
    plt.ylim([-180, 180])
    plt.yticks(np.arange(-180, 181, 60))
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Phase (o)')
    plt.tight_layout() 

iPyWid.interactive( plot, 
                    filtType=filtTypeDropdown,
                    fc=fcSlider,
                    gainIndB=gainSlider, 
                    Q=qSlider, 
                    fs=fsSlider)

interactive(children=(Dropdown(description='Filter Type', index=4, options=('LowPass', 'HighPass', 'HighShelf'…